In [8]:
import sys
import argparse

import neuroglancer
import neuroglancer.cli
import numpy as np
import nibabel as nib

In [10]:
def read_nifti_file(file_path):
    nifti = nib.load(file_path)
    data = nifti.get_fdata()
    affine = nifti.affine
    print(f"Shape of the data from {file_path}: {data.shape}")
    print(f"Affine of the data from {file_path}:\n{affine}")
    return data, affine

# Paths to your NIfTI files
angio_file = '../data/angio.nii.gz'
fusi_file = '../data/fusi.nii'

data1, affine1 = read_nifti_file(angio_file)
data2, affine2 = read_nifti_file(fusi_file)


pixdim[1,2,3] should be positive; setting to abs of pixdim values
Level 35:nibabel.global:pixdim[1,2,3] should be positive; setting to abs of pixdim values


Shape of the data from ../data/angio.nii.gz: (147, 200, 135)
Affine of the data from ../data/angio.nii.gz:
[[  0.2         -0.          -0.         -14.60000038]
 [ -0.           0.2         -0.         -14.40000057]
 [  0.           0.           0.2         -3.60000014]
 [  0.           0.           0.           1.        ]]
Shape of the data from ../data/fusi.nii: (192, 151, 173)
Affine of the data from ../data/fusi.nii:
[[ -0.11         0.           0.          10.50499994]
 [  0.           0.2          0.         -15.00000022]
 [  0.           0.           0.09856     -8.47615983]
 [  0.           0.           0.           1.        ]]


In [13]:
def create_coordinate_space(affine):
    scales = np.sqrt((affine[:3, :3] ** 2).sum(axis=0))
    scales = np.abs(scales).tolist()
    print(f"Scales: {scales}")
    return neuroglancer.CoordinateSpace(
        names=["x", "y", "z"],
        units=["mm", "mm", "mm"],
        scales=scales
    )

dimensions1 = create_coordinate_space(affine1)
dimensions2 = create_coordinate_space(affine2)

Scales: [0.20000000298023224, 0.20000000298023224, 0.20000000298023224]
Scales: [0.10999999940395355, 0.20000000298023224, 0.09855999797582626]


In [14]:
# Determine a unified scale for visualization, taking the minimum scale for each dimension
unified_scales = [min(dimensions1.scales[i], dimensions2.scales[i]) for i in range(3)]
unified_dimensions = neuroglancer.CoordinateSpace(
    names=["x", "y", "z"],
    units=["mm", "mm", "mm"],
    scales=unified_scales
)

print(f"Unified scales: {unified_scales}")

Unified scales: [0.00010999999940395356, 0.00020000000298023223, 9.855999797582626e-05]


In [15]:
if __name__ == "__main__":
    if 'ipykernel_launcher' in sys.argv[0]:
        sys.argv = [sys.argv[0]]  # Ignore notebook arguments
    
    ap = argparse.ArgumentParser()
    neuroglancer.cli.add_server_arguments(ap)
    args = ap.parse_args()
    neuroglancer.cli.handle_server_arguments(args)

    viewer = neuroglancer.Viewer()

    with viewer.txn() as s:
        s.dimensions = unified_dimensions  # Use unified dimensions
        
        s.layers["nifti1"] = neuroglancer.SegmentationLayer(
            source=[
                neuroglancer.LocalVolume(data=data1.astype(np.uint32), dimensions=unified_dimensions)
            ],
        )

        s.layers["nifti2"] = neuroglancer.SegmentationLayer(
            source=[
                neuroglancer.LocalVolume(data=data2.astype(np.uint32), dimensions=unified_dimensions)
            ],
        )

    print(viewer)

http://127.0.0.1:64140/v/9a3821912b0f11635c24225da4d100f3ab3a2b18/
